In [ ]:
import glob, os, json
import solver
import pickle

import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score

import time
import math

import traceback

In [ ]:
def generateData(rows, labels):
    inputs = np.empty(shape=(rows, labels))
    ordering = np.empty(shape=(rows, labels))
    for i in range(rows):
#         perm = np.random.permutation(np.arange(6, 6 + labels))
        perm = np.random.permutation(labels)
#         number = 0
#         for j in range(labels):
#             number += perm[j] * math.pow(10, labels - j - 1)
            
        ordering[i] = perm
        inputs[i] = perm[::-1]

#         inputs[i, 0] = number
#         inputs[i, 1] = 0
        
    return (inputs, ordering)

inputs, outputs = generateData(10000, 5)

In [ ]:
inputs = normalize(inputs)
unnormalizedOutputs = outputs
outputs = normalize(outputs)

In [ ]:
size = inputs.shape[0]
# Test data is separated in cleaning stage
trainSize = int(size * 0.75)
validSize = size - trainSize

inputsTrain = inputs[0:trainSize]
outputsTrain = outputs[0:trainSize]

inputsValid = inputs[trainSize:]
outputsValid = outputs[trainSize:]
unnormalizedOutputsValid = unnormalizedOutputs[trainSize:]

In [ ]:
with tf.Session().as_default():
    array1 = tf.constant([ 1,  3,  2,  0,  4], dtype=tf.float64)
    array2 = tf.constant([ 3,  2,  0,  4,  1], dtype=tf.float64)
    print((spearmanCorrelation(array1, array2).eval() + 1)/2)

In [ ]:
with tf.Session().as_default():
#     print(outputsTrain[])
#     print(tf.reduce_mean(ndcg(outputsTrain, outputsTrain)).eval())
    c = tf.map_fn(lambda x: boundedSpearman(x[0], x[1]), (outputsTrain, outputsTrain), dtype=tf.float64)
    print(tf.reduce_mean(c).eval())
#     print(tf.gather(outputsTrain, tf.nn.top_k(outputsTrain, k=5).indices).eval())

In [ ]:
EPOCHS = 10000

N1 = trainSize
LABEL_COUNT = 5

NODES1 = 512
NODES2 = 256

ALPHA = 0.08

BATCH_SIZE = 30

STD = 0.1

LEARNING_RATE = 0.1

In [ ]:
# Define the input function for training
inputFunc = tf.estimator.inputs.numpy_input_fn(
    x={"input": inputsTrain}, y=outputsTrain,
    batch_size=BATCH_SIZE, num_epochs=EPOCHS, shuffle=True)

In [ ]:
# Define the neural network
def network(xDict):
    x = xDict["input"]
    
    flatten = tf.contrib.layers.flatten(x)
        
    regularizer = tf.contrib.layers.l2_regularizer(scale=ALPHA)
    
    # Hidden fully connected layer
#     layer1 = tf.layers.dense(flatten, NODES1, kernel_regularizer=regularizer, activation=tf.nn.relu)
    # Output fully connected layer with a neuron for each class
    
    layer1 = tf.layers.dense(flatten, NODES1, kernel_regularizer=regularizer, activation=tf.nn.relu)
    # Hidden fully connected layer
    layer2 = tf.layers.dense(flatten, 64, kernel_regularizer=regularizer, activation=tf.nn.relu)
    # Output fully connected layer with a neuron for each class
    outLayer = tf.layers.dense(layer2, LABEL_COUNT)
    return outLayer

In [ ]:
# Kullback-Leibler Divergence, as per https://stackoverflow.com/a/43298483
def klDivergence(p, q):
    pClipped = tf.clip_by_value(p, 1e-10, 1.0)
    qClipped = tf.clip_by_value(q, 1e-10, 1.0)
    return tf.reduce_sum(pClipped * tf.log(pClipped/qClipped))

# Loss function based off of Jensen-Shannon Divergence
def loss(label, prediction):
    mean = 0.5 * (label + prediction)
    return 0.5 * klDivergence(label, mean) + 0.5 * klDivergence(prediction, mean)

def log2(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(2, dtype=numerator.dtype))
    return numerator / denominator

# Accuracy metric using Normalized Discounted Cumulative Gain, as per https://github.com/shiba24/learning2rank/
def ndcg(labels, predictions, k=5):
    topK = tf.nn.top_k(labels, k=5)
    sortedValues = topK.values
    sortedIndices = topK.indices
#         print(labelSorted)
#         labelSorted = sorted(label, reverse=True)
    ideal_dcg = 0
    for i in range(k):
#             ideal_dcg += (2 ** labelSorted[:i] - 1.) / log2(tf.cast(i + 2, tf.float64))
        ideal_dcg += (sortedValues[i] + 1) / log2(tf.cast(i + 2, tf.float64))
    dcg = 0
#         argsort_indices = np.argsort(predictions)[::-1]
#         argsort_indices = tf.nn.top_k(predictions, k=5).indices
#         print(argsort_indices)
    for i in range(k):
        dcg += (tf.gather(predictions, sortedIndices[i]) + 1) / log2(tf.cast(i + 2, tf.float64))
#         dcg += (predictions[i] + 1) / log2(tf.cast(i + 2, tf.float64))
    return dcg / ideal_dcg

def spearmanCorrelation(label, prediction):
    length = tf.cast(tf.shape(prediction)[0], tf.float64)
    sumVal = tf.reduce_sum(tf.square(tf.subtract(prediction, label)))
    return 1 - 6 * sumVal / (length ** 3 - length)

# Bound Spearman coeff. between 0 and 1
def boundedSpearman(label, prediction):
    return (spearmanCorrelation(label, prediction) + 1.)/2

# Builds an integer ranking out of a 1-D tensor
def convertPredToRank(prediction):
    return tf.cast(tf.nn.top_k(prediction, k=5).indices, dtype=tf.float64)

In [ ]:
# Define the model function (following TF Estimator Template)
def modelFunc(features, labels, mode):
    # Build the neural network
    logits = network(features)
    
#     resizedLogits = tf.reshape(logits, shape=[-1, MAX_SIZE * MAX_SIZE, 1])
    
    # Predictions
    # TODO: Possibly need to change
#     pred_classes = logits
    pred_classes = tf.map_fn(convertPredToRank, logits)
#     pred_classes = tf.argmax(logits, axis=1)
#     pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
    # Define loss and optimizer
#     loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#         logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
#     loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
#         logits=logits, labels=labels))
    loss_op = tf.reduce_mean(loss(labels, logits))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
#     acc_op = tf.metrics.accuracy(labels=tf.argmax(labels, axis=1), predictions=pred_classes)
#     acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    ndcg_map = tf.map_fn(lambda x: boundedSpearman(x[0], x[1]), (labels, pred_classes), dtype=tf.float64)
    acc_op = tf.metrics.mean(ndcg_map)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [ ]:
# Build the Estimator
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
model = tf.estimator.Estimator(modelFunc, config=tf.contrib.learn.RunConfig(session_config=config))

In [ ]:
# Train the Model
model.train(inputFunc, steps=1)

In [ ]:
# Evaluate the Model
# Define the input function for evaluating
validFunc = tf.estimator.inputs.numpy_input_fn(
    x={"input": inputsValid}, y=unnormalizedOutputsValid,
    batch_size=BATCH_SIZE, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(validFunc)